In [92]:
import numpy as np
import cv2
import sys

from bird_pose_tracking.trackPosture_slp import results

sys.path.append("C:/Users/xl313/OneDrive/Documents/GitHub/poseTrackingXL")
sys.path.append("C:/Users/xl313/OneDrive/Documents/GitHub/poseTrackingXL/utils")
from load_matlab_data import loadmat_sbx
sys.path.append("C:/Users/xl313/OneDrive/Documents/GitHub/poseTrackingXL/camera_calibration")
from pySBA import PySBA
import matplotlib.pyplot as plt

Using matplotlib backend: QtAgg


<module 'notebook' from 'C:\\Users\\xl313\\anaconda3\\envs\\sleap\\lib\\site-packages\\notebook\\__init__.py'>

In [193]:
''' Data Params '''
# define file paths
bird_id = 'RBY52'
session_date = '2ndPart_012425'
session_root = f'Z:/Sherry/acquisition'
session_dir = f'{bird_id}_{session_date}'
vid_path = f'{session_root}/{session_dir}/'
pred_file = f'032325_posture_face.npy'
pred_path = f"{vid_path}{pred_file}"
print(pred_path)
# to save training images
save_dir = 'Z:/Sherry/poseTrackingXL/SeedCarryingLabeling/LabeledData/'
save_file = f'seedLabel_{session_dir}.npz'
save_path = f'{save_dir}{save_file}'

# camera params
cam_params = loadmat_sbx("Z:/Sherry/poseTrackingXL/calibration_files/all_opt_arrays/102324_negated_camParams")['camParams_negateR']
cam_ids = ['blue_cam', 'green_cam', 'red_cam', 'yellow_cam'] # make sure it is in order 
n_cams = len(cam_ids)

# video params
n_images = 50 # 1000 # number of frames to label


Z:/Sherry/acquisition/RBY52_2ndPart_012425/032325_posture_face.npy
Z:/Sherry/poseTrackingXL/calibration_files/all_opt_arrays/102324_negated_camParams


In [181]:
''' Get the frame index - random frames near interactions '''
# load the site interactions
annotated_seed_path = f'{vid_path}/annotatedSeeds.mat'
seed_struct = loadmat_sbx(annotated_seed_path)['annotatedSeeds']
count_data = seed_struct['countData']

# get interaction frames
all_int_start = count_data['newSite']
all_int_end = count_data['endSite']

# take surrounding frames
int_frames = np.concatenate((all_int_start - 5, all_int_end + 5))
near_idx = np.arange(6, 15)
for i in near_idx:
    int_frames = np.concatenate((int_frames, all_int_start-i))
    int_frames = np.concatenate((int_frames, all_int_end+i))

# get caches + retrievals frames
all_int_changes = np.sum(seed_struct['seedChanges'], axis=1) 
cache_onsets = all_int_start[all_int_changes > 0]
cache_offsets = all_int_end[all_int_changes > 0]
ret_onsets = all_int_start[all_int_changes < 0]
ret_offsets = all_int_end[all_int_changes < 0]

# take more surrounding frames
near_idx = np.arange(15, 50)
for i in near_idx:
    int_frames = np.concatenate((int_frames, cache_onsets - i, cache_offsets + i,
                                 ret_onsets - i, ret_offsets + i))

# select random frames near interactions to label
frame_idx = np.random.choice(int_frames, size=n_images, replace=False)
frame_idx = np.sort(frame_idx)

Z:/Sherry/acquisition/RBY52_2ndPart_012425//annotatedSeeds.mat


In [182]:
''' For cropping '''
def crop_from_com(img, centroid, half_width, crop_size = (320,320)): # used to be (320,320)
    '''
    Crops an image around a given centroid (crop dims defined by half_width)
    and resizes to the specified crop_size.
    '''
    ctr = np.round(centroid).astype(int)
    half_width = np.round(half_width).astype(int)
    img_h,img_w = img.shape[0],img.shape[1] # used to be img_h, img_w = img.shape
    
    xmin = np.min([np.max([ctr[0] - half_width, 0]), img_w - 1])
    xmax = np.max([np.min([ctr[0] + half_width + 1, img_w]), 1])
    ymin = np.min([np.max([ctr[1] - half_width, 0]), img_h - 1])
    ymax = np.max([np.min([ctr[1] + half_width + 1, img_h]), 1])
    
    cropped_img = img[ymin:ymax, xmin:xmax]
    
    crop_img = cv2.resize(cropped_img, crop_size, interpolation=cv2.INTER_AREA)
    min_ind = np.array([xmin, ymin])
    max_ind = np.array([xmax, ymax])
    crop_scale = crop_size / (max_ind - min_ind)
    
    return crop_img, min_ind, crop_scale, 

face_crop_size = (128,128) # pixels
face_w3d = 20 # scaling factor # IL: 0.08
com_head_ind = 0
face_images = np.full((face_crop_size[1], face_crop_size[0], 3, n_cams, n_images), 0, dtype='uint8') # added 3

In [194]:
''' Create image set '''
# load the COM predictions to find the face
print('Loading COM predictions')
results_dict = np.load(pred_path, allow_pickle=True)

Loading COM predictions


In [198]:
#results = results_dict['results']
results_dict

array({'results': {'posture_preds': array([[[  45.39491619,  224.86052463,   12.36436414],
        [  45.40077151,  224.87219966,   12.49223672],
        [  37.54122964,  225.47887722,   19.39251366],
        ...,
        [  38.31895959,  244.20587765,   36.40905781],
        [  49.78042194,  240.85467195,   10.04110312],
        [  46.26114183,  224.49596195,   10.56360672]],

       [[  45.38371495,  224.87300042,   12.37021748],
        [  45.38862535,  224.8848558 ,   12.49760718],
        [  37.81950734,  225.23561733,   19.38505557],
        ...,
        [  38.44652332,  244.13520902,   36.61034086],
        [  55.46885504,  228.58041798,   11.44152345],
        [  46.26084143,  224.50827639,   10.56727278]],

       [[  45.38374709,  224.8673054 ,   12.36628106],
        [  45.3910762 ,  224.87714819,   12.49135888],
        [  37.80415245,  225.2342319 ,   19.38204162],
        ...,
        [  38.43971884,  244.10081373,   36.62238061],
        [  55.48262455,  228.59910934,   

In [186]:
com_preds = results['com_preds'] # shape (n_frames, n_keypoints, 3)
n_frames_total = com_preds.shape[0] # total number of frames in video

# define the video reader for each camera
all_readers = []
for i in range(len(cam_ids)):
    cam = cam_ids[i]
    print(cam)
    camPath = f"{vid_path}{cam}.avi"

    # define the video reader obj and settings
    api_id = cv2.CAP_FFMPEG
    reader = cv2.VideoCapture(camPath, api_id)
    all_readers.append(reader)

# flag that video reading failed
stopReading = False

blue_cam
green_cam
red_cam
yellow_cam


In [187]:
# crop and save each frame
print('Saving cropped images')
sba = PySBA(cam_params, np.nan, np.nan, np.nan, np.nan)
im_idx = 0
all_min_ind = np.zeros((2, 4, len(frame_idx)))
ctr = np.zeros((2,4,len(frame_idx)))
# Collect all full images in a dictionary
all_full_imgs = {nCam: [] for nCam in range(n_cams)}

for n_frame in range(n_frames_total):
    # Read and downsample
    full_img = []
    for nCam in range(n_cams):
        if n_frame in frame_idx:
            all_readers[nCam].set(cv2.CAP_PROP_POS_FRAMES, n_frame)
            flag, img = all_readers[nCam].read()
            if img is None:
                stopReading = True
                break
            full_img.append(img)  # used to be img[:,:,0]
            if full_img[nCam] is None:
                stopReading = True
                break
    # If reading for any video failed, terminate tracking
    if stopReading:
        print(f'Terminated Reading on Frame {n_frame}')
        break

    # Append full images to the dictionary
    if full_img:
        for nCam, img in enumerate(full_img):
            all_full_imgs[nCam].append(img)

    # Proceed with cropping and saving cropped images
    if n_frame in frame_idx:
        print(f'Reading Frame {im_idx + 1}/{n_images}')
        # get the 3D distance from each camera for cropping scale
        head_COM = com_preds[n_frame, com_head_ind]
        head_reproj = sba.project(np.tile(head_COM, (n_cams, 1)),
                                  cam_params)  # get reprojected body centroid location for each camera
        camDist = sba.rotate(np.tile(head_COM, (n_cams, 1)),
                             cam_params[:, :3])  # rotate to camera coordinates
        camDist = camDist[:, 2] + cam_params[:, 5]  # get z-axis distance ie along optical axis
        camScale = cam_params[:, 6] / camDist  # convert to focal length divided by distance
        half_width = camScale * face_w3d

        # save the cropped image for each camera
        min_ind = np.full((n_cams, 1, 2), np.nan)
        crop_scale = np.full((n_cams, 1, 2), np.nan)
        for nCam in range(n_cams):
            thisCom = np.maximum(head_reproj[nCam], 0)
            thisCom[0] = np.minimum(thisCom[0], full_img[nCam].shape[1])  # x limit is shape[1]
            thisCom[1] = np.minimum(thisCom[1], full_img[nCam].shape[0])  # y limit is shape[0]
            ctr[:,nCam,im_idx] = thisCom
            thisHalfWidth = np.maximum(half_width[nCam], 15)  # minimum 31px image for head
            face_images[:, :, :, nCam, im_idx], all_min_ind[:, nCam, im_idx], _ = crop_from_com(full_img[nCam],
                                                                                                thisCom,
                                                                                                thisHalfWidth,
                                                                                                face_crop_size)
        im_idx += 1
    if im_idx == n_images:
        break

# Output all full images as a dictionary
print("All full images are stored in the variable `all_full_imgs`.")

Saving cropped images
Reading Frame 1/50
Reading Frame 2/50
Reading Frame 3/50
Reading Frame 4/50
Reading Frame 5/50
Reading Frame 6/50
Reading Frame 7/50
Reading Frame 8/50
Reading Frame 9/50
Reading Frame 10/50
Reading Frame 11/50
Reading Frame 12/50
Reading Frame 13/50
Reading Frame 14/50
Reading Frame 15/50
Reading Frame 16/50
Reading Frame 17/50
Reading Frame 18/50
Reading Frame 19/50
Reading Frame 20/50
Reading Frame 21/50
Reading Frame 22/50
Reading Frame 23/50
Reading Frame 24/50
Reading Frame 25/50
Reading Frame 26/50
Reading Frame 27/50
Reading Frame 28/50
Reading Frame 29/50
Reading Frame 30/50
Reading Frame 31/50
Reading Frame 32/50
Reading Frame 33/50
Reading Frame 34/50
Reading Frame 35/50
Reading Frame 36/50
Reading Frame 37/50
Reading Frame 38/50
Reading Frame 39/50
Reading Frame 40/50
Reading Frame 41/50
Reading Frame 42/50
Reading Frame 43/50
Reading Frame 44/50
Reading Frame 45/50
Reading Frame 46/50
Reading Frame 47/50
Reading Frame 48/50
Reading Frame 49/50
Reading

In [164]:
''' plot a frame to check output '''
ex_frame = 1
# fig params
f, ax = plt.subplots(2, 2)
colors = ['xkcd:scarlet']
# plot for each camera
ax[0, 0].imshow(all_full_imgs.get(0)[ex_frame]) 
ax[0, 0].scatter(ctr[0, 0, ex_frame],ctr[1, 0, ex_frame], c= 'r', marker='*', s=15)
ax[0, 1].imshow(all_full_imgs.get(1)[ex_frame]) 
ax[0, 1].scatter(ctr[0, 1, ex_frame],ctr[1, 1, ex_frame], c= 'r', marker='*', s=15)
ax[1, 0].imshow(all_full_imgs.get(2)[ex_frame])
ax[1, 0].scatter(ctr[0, 2, ex_frame],ctr[1, 2, ex_frame], c= 'r', marker='*', s=15)
ax[1, 1].imshow(all_full_imgs.get(3)[ex_frame]) #
ax[1, 1].scatter(ctr[0, 3, ex_frame],ctr[1, 3, ex_frame], c= 'r', marker='*', s=15)

plt.show()
plt.tight_layout()

In [188]:
face_images_rgb = np.zeros_like(face_images)  # Initialize an array to store RGB images
for cam in range(face_images.shape[3]):  # Iterate over cameras
    for img_idx in range(face_images.shape[4]):  # Iterate over images
        face_images_rgb[:, :, :, cam, img_idx] = cv2.cvtColor(face_images[:, :, :, cam, img_idx], cv2.COLOR_BGR2RGB)

# # Display images from all cameras for a specific frame (e.g., the 3rd frame in this example)
# selected_frame_idx = 9  # Change to the frame number you'd like to visualize
# fig, axs = plt.subplots(1, face_images.shape[3], figsize=(15, 5))  
# 
# for cam in range(face_images.shape[3]):    
#     axs[cam].imshow(face_images_rgb[:, :, :, cam, selected_frame_idx])  
#     axs[cam].set_title(f'Camera {cam + 1}')
#     axs[cam].axis('off')
# 
# plt.tight_layout()
# plt.show()

In [189]:
def label_images(images):
    seed_labels = []
    # user input - seed labels
    def on_key(event):
        if event.key in ['0', '1']:
            print(f"Key pressed: {event.key}")
            seed_labels.append(int(event.key))
            idx = len(seed_labels)
            if idx < images.shape[-1]:
                for ax_row in ax:  # Iterate over each subplot row
                    for a in ax_row:  # Iterate over individual plots
                        a.clear()
                # present the next image
                ax[0, 0].imshow(images[:, :, :, 0, idx], cmap='gray')
                ax[1, 0].imshow(images[:, :, :, 1, idx], cmap='gray')
                ax[0, 1].imshow(images[:, :, :, 2, idx], cmap='gray')
                ax[1, 1].imshow(images[:, :, :, 3, idx], cmap='gray')
                ax[2, 0].imshow(all_full_imgs.get(0)[idx]) 
                ax[2, 0].scatter(ctr[0, 0, idx],ctr[1, 0, idx], c= 'r', marker='*', s=15)
                ax[3, 0].imshow(all_full_imgs.get(1)[idx]) 
                ax[3, 0].scatter(ctr[0, 1, idx],ctr[1, 1, idx], c= 'r', marker='*', s=15)
                ax[2, 1].imshow(all_full_imgs.get(2)[idx])
                ax[2, 1].scatter(ctr[0, 2, idx],ctr[1, 2, idx], c= 'r', marker='*', s=15)
                ax[3, 1].imshow(all_full_imgs.get(3)[idx]) #
                ax[3, 1].scatter(ctr[0, 3, idx],ctr[1, 3, idx], c= 'r', marker='*', s=15)
                fig.suptitle(f"Image {idx+1}/{images.shape[-1]}: Press 1 (seed in beak) or 0 (no seed)")
                fig.canvas.draw()
            else:
                print("No more images to process.")
                plt.close()
        else:
            print("Invalid key pressed. Please press either '0' or '1'.")

    # display image
    fig, ax = plt.subplots(4, 2)
    ax[0, 0].imshow(images[:, :, :, 0, 0], cmap='gray') # first cam
    ax[1, 0].imshow(images[:, :, :, 1, 0], cmap='gray')
    ax[0, 1].imshow(images[:, :, :, 2, 0], cmap='gray')
    ax[1, 1].imshow(images[:, :, :, 3, 0], cmap='gray')
    ax[2, 0].imshow(all_full_imgs.get(0)[0]) 
    ax[2, 0].scatter(ctr[0, 0, 0],ctr[1, 0, 0], c= 'r', marker='*', s=15)
    ax[2, 1].imshow(all_full_imgs.get(1)[0]) 
    ax[2, 1].scatter(ctr[0, 1, 0],ctr[1, 1, 0], c= 'r', marker='*', s=15)
    ax[3, 0].imshow(all_full_imgs.get(2)[0])
    ax[3, 0].scatter(ctr[0, 2, 0],ctr[1, 2, 0], c= 'r', marker='*', s=15)
    ax[3, 1].imshow(all_full_imgs.get(3)[ex_frame]) #
    ax[3, 1].scatter(ctr[0, 3, 0],ctr[1, 3, 0], c= 'r', marker='*', s=15)
    fig.suptitle(f"Image 1/{images.shape[-1]}: Press 1 (seed in beak) or 0 (no seed)")
    fig.canvas.mpl_connect('key_press_event', on_key)
    plt.show()

    return np.asarray(seed_labels)
seed_labels = label_images(face_images_rgb)

In [91]:
# save as a dict
save_dict = {
    "all_images": face_images,
    "seed_labels": seed_labels,
    "color_labels": color_labels
}
print(f'Saving file to {save_path}')
np.savez_compressed(f'{save_path}', **save_dict)

Using matplotlib backend: QtAgg
Saving file to Z:/Sherry/poseTrackingXL/SeedCarryingLabeling/LabeledData/seedLabel_AMB155_031025.npz


In [102]:
test = np.load("Z:/Sherry/poseTrackingXL/SeedCarryingLabeling/LabeledData/seedLabel_AMB155_031025.npz")
list(test.keys())
test['color_labels']

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 3])